### In this notebook we calculate the statistics of the existing datasets and the Wiki_TabNER dataset. The final table is at the end of the notebook

In [18]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
import json
import spacy
import tqdm
import csv
import os
from scipy.stats import kurtosis

In [2]:
import glob

In [1]:
import numpy as np
from scipy import stats
import math

##### To download the spacy model, use the following command: python -m spacy download en_core_web_sm

In [3]:
nlp = spacy.load('en_core_web_sm')

In [5]:
cell = 'Unforgiven – Clint Eastwood A Few Good Men – Rob Reiner , Andrew Scheinman Howards End – Ismail Merchant Scent of a Woman – Martin Brest The Crying Game – Stephen Woolley'

In [6]:
len([tok for tok in nlp(str(cell))])

32

### Wikitables analysis

We first analyse the WikiTables corpus. We calculate the number of links per cell in the tables. We want to extract how many of the total number of cells in the WikiTables corpus, have 2 or more links.

In [7]:
original_wikitables = "../data/original_dataset/tables.json"
wiki_links_ids = "../data/original_dataset/wiki_links-random.txt"

In [8]:
# Function to analyze entity counts in a batch
def analyze_batch(batch_data):
    entity_counts = []    
    
    for table in batch_data:
        table_data = table['tableData']
       
        for row in table_data:
            for cell in row:       
                if len(cell["surfaceLinks"])>0:
                    entity_counts.append(len(cell["surfaceLinks"]))
                else:
                    entity_counts.append(0)
                    
    return pd.Series(entity_counts)

In [ ]:
with open("../data/original_dataset/tables.json", "r") as file:

    batch_size = 1500000  
    batch_data = [json.loads(next(file)) for _ in range(batch_size)]
    total_entity_counts = pd.Series()

    batch_entity_counts = analyze_batch(batch_data)
    total_entity_counts = pd.concat([total_entity_counts, batch_entity_counts])
    
total_entity_counts.to_csv("wiki_tables_entity_count.csv")
stats_all_batches = total_entity_counts.describe()

print("Summary Statistics on Entity Counts for All Batches:")
print(stats_all_batches)

In [11]:
total_entity_counts = pd.read_csv("../wiki_tables_entity_count.csv")

In [12]:
len(total_entity_counts)

93957683

In [13]:
column_names = ["cell", "num_ent"]
total_entity_counts.columns = column_names
del total_entity_counts["cell"]

In [14]:
no_zeros=total_entity_counts.loc[total_entity_counts.num_ent>0]

In [15]:
len(no_zeros) # 25 percent of all cells have at least 1 link, 4% of all have at least 2

24206485

In [16]:
no_zeros['num_ent'].describe()

count    2.420648e+07
mean     1.319904e+00
std      2.544662e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      9.170000e+02
Name: num_ent, dtype: float64

In [17]:
more_than_1=no_zeros.loc[no_zeros.num_ent>2]

In [18]:
len(more_than_1) # 3.7 percent of all cells have more than 2 links per cell.

896039

### Other datasets analysis

The T2D, Limaye and WikiGS datasets are downloaded from http://www.cs.toronto.edu/~oktie/webtables/

In [ ]:
t2d_path = "../Efthymiou_datasets/webtables-evaluation-data/csv/t2d_tables_instance/"
limaye_path = "../Efthymiou_datasets/webtables-evaluation-data/csv/LimayeGS/tables_instance/"
wiki_gold_standard_path = "../Efthymiou_datasets/webtables-evaluation-data/csv/WikipediaGS/tables_instance/csv/"

In [ ]:
git_path = "../data/other_data/Git_tables/"

In [ ]:
sato_path = "../data/other_data/sato_tables/all/K0/" #itterate here over the K splits and calc avg stats

In [ ]:
# random selection of 15000 tables for analysis
dfs_wiki = load_tables(wiki_gold_standard_path)
dfs_wiki = dfs_wiki[100000:115000]

In [19]:
def load_tables(folder_path):
    dfs = []
    csv_files = [f for f in os.listdir(folder_path) ]
    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        try:
            df = pd.read_csv(file_path, engine='python', on_bad_lines='skip')
            dfs.append(df)
        except pd.errors.EmptyDataError as e:
                print(f"No data found in CSV file: {csv_file}")
    return dfs

In [44]:
def compute_stats(dfs):
    
    stats = {"mean_num_rows":0, "std_num_rows":0, "mean_num_cols":0, "std_num_cols":0, "mean_num_toks":0, "std_num_toks":0, "kurtosis":0}
    overall_num_rows = []
    overall_num_cols = []
    overall_num_toks = []
    overall_info_dens = []
    num_ents_comma = []
    
    for df in dfs:
        
        named_ents = named_entities_cell[i]
        
        num_cols = len(df.columns)
        num_rows = len(df)

        overall_num_rows.append(num_rows)
        overall_num_cols.append(num_cols)

        for index,row in df.iterrows():
            for col in df.columns:  
             
                cell_text = row[col]
                
                ents_comma = str(cell_text).split(",")
                num_ents_comma.append(len(ents_comma))

                num_tokens = len([tok for tok in nlp(str(cell_text))])            
                overall_num_toks.append(num_tokens)
                
    stats["mean_num_rows"] = np.mean(overall_num_rows)
    stats["std_num_rows"] = np.std(overall_num_rows)

    stats["mean_num_cols"] = np.mean(overall_num_cols)
    stats["std_num_cols"] = np.std(overall_num_cols)
    
    stats["mean_num_toks"] = np.mean(overall_num_toks)
    stats["std_num_toks"] = np.std(overall_num_toks)

    stats["mean_comma"] = np.mean(num_ents_comma)
    stats["std_comma"] = np.std(num_ents_comma)
        
    return stats


In [21]:
def add_stats(dataset_name, file_path, final_stats_df):    
        
    #dfs = load_tables(file_path)    
    print(len(dfs))
    stats = compute_stats(dfs)
    
    stats_df =  pd.DataFrame([stats])
    stats_df["dataset"] = dataset_name
    stats_df["num_tables"] = len(dfs)
    
    df_final = pd.concat([final_stats_df, stats_df], ignore_index=True)
    
    df_final.to_csv("overall_dataset_stats.csv")
    
    return df_final

In [26]:
# load current stats to add new 
current_stats = pd.read_csv("overall_dataset_stats.csv")

In [ ]:
added_wikiGS = add_stats("WikiGS", wiki_gold_standard_path, added_sato0)

In [ ]:
# Computing the mean over the data split from SATO

only_sato = added_sato0[added_sato0['dataset'].str.contains("sato")]
mean_row = only_sato[numeric_columns].mean(axis=0)
mean_row['num_tables'] = only_sato['num_tables'].sum(axis=0)
mean_row['dataset'] = "sato_multi"

In [ ]:
added_sato0.loc[12] = mean_row

### TURL entity linking eval dataset

The link for donloading the TURL data can be found in their github repository: https://github.com/sunlab-osu/TURL

In [ ]:
# similar for the test_own.table_entity_linking.json
with open(os.path.join('../turl_dataset', 'dev.table_entity_linking.json'), 'r') as f:
    turl_dataset = json.load(f)

In [ ]:
added_turl = add_stats("turl_el_dev", turl_dataset, current_stats)

In [ ]:
turl_el = added_turl.loc[added_turl['dataset']=="turl_el"]
turl_el_dev= added_turl.loc[added_turl['dataset']=="turl_el_dev"]

In [ ]:
# Computing the mean over the dev and test data from TURL
merged_df = pd.concat([turl_el, turl_el_dev])
mean_row = merged_df[numeric_columns].mean(axis=0)

turl_mean = mean_row.to_dict()
turl_mean['num_tables'] = 10927 # this is the sum of the dev and test tables
turl_mean['dataset'] = "turl_el_test_dev"

In [ ]:
added_turl.loc[7] = turl_mean

In [ ]:
added_turl.to_csv("overall_dataset_stats.csv")

### Wiki_TabNER analysis

In [19]:
tabner_data_path="../data/Wiki_TabNER_final_labeled.json"
with open(tabner_data_path, 'r') as f:
    ner_tables = json.load(f)  

In [20]:
dfs = []
named_entities_cell = []
for i in range(len(ner_tables)):
  
    tableHeaders = ner_tables[i][0][4]
    table_data = ner_tables[i][0][5]    
    
    named_entities_cell.append(ner_tables[i][0][6])
    
    columns = [tableHeaders[i] for i in range(len(tableHeaders))]
 
    row_indexes = [item[0][0] for item in table_data]
    col_indexes = [item[0][1] for item in table_data]
    values = [item[1] for item in table_data]

    # Create a dictionary to hold the data
    data_dict = {}
    for row_idx, col_idx, value in zip(row_indexes, col_indexes, values):
        if row_idx not in data_dict:
            data_dict[row_idx] = {}
        data_dict[row_idx][col_idx] = value

    df = pd.DataFrame.from_dict(data_dict, orient='index')
   
    df.index.name = None
    dfs.append(df)

In [24]:
added_tabner = add_stats("wiki_tabner_final", tabner_data_path, current_stats)

61273


### Final datasets statistics

In [42]:
current_stats = pd.read_csv("overall_dataset_stats.csv")
current_stats.drop('Unnamed: 0', axis=1, inplace=True)

In [43]:
current_stats.sort_values(["dataset"])

,dataset,num_tables,mean_num_rows,std_num_rows,mean_num_cols,std_num_cols,mean_num_toks,std_num_toks,mean_comma,std_comma
12,WikiGS,15000,13.783733,35.450495,4.578733,3.356179,2.471672,5.704536,1.103979,0.618069
3,git_tables,1101,58.196185,94.693164,16.873751,11.600740,1.868231,10.062218,1.037563,0.617178
1,limaye,428,34.549065,40.992761,3.787383,1.233917,1.706675,1.392773,1.033316,0.182750
10,sato_K0,15736,17.848310,60.938394,1.530249,0.787049,2.936658,9.177716,1.123136,1.106772
4,sato_K1,15754,18.480640,75.620203,1.531801,0.785103,2.995293,8.307041,1.127742,0.741978
9,sato_K2,15776,17.669118,54.755346,1.531757,0.796962,2.957975,8.619747,1.130033,0.939588
7,sato_K3,15714,18.481991,77.590398,1.531437,0.801749,3.033713,8.182333,1.127927,0.984825
8,sato_K4,15753,17.897162,55.324801,1.534120,0.783969,2.924267,7.192327,1.117836,0.568125
11,sato_multi,78733,18.075444,64.845828,1.531873,0.790966,2.969581,8.295833,1.125335,0.868258
0,t2d,233,121.600858,115.509022,4.948498,1.793386,1.913527,1.791974,1.000000,0.000000


### Calculate Welch's T-test

In [2]:
def calc_mean_toks(dfs):
    overall_num_toks = []     
    for df in dfs:       

        for index,row in df.iterrows():
            for col in df.columns:               
                cell_text = row[col]     
                
                num_tokens = len([tok for tok in nlp(str(cell_text))])            
                overall_num_toks.append(num_tokens)
                
    return overall_num_toks

In [ ]:
tabner_mean_toks = calc_mean_toks(dfs)

In [8]:
tabner_mean_toks = np.load('tabner_mean_toks.npy')
print(len(tabner_mean_toks))

4358559


In [9]:
wikigs_mean_toks = np.load('wikigs_mean_toks.npy')
print(len(wikigs_mean_toks))

1041715


In [14]:
stats.ttest_ind(tabner_mean_toks, wikigs_mean_toks, equal_var=False)

TtestResult(statistic=223.8981546350554, pvalue=0.0, df=4713311.415947707)

In [15]:
tabner_mean_toks_smaller = tabner_mean_toks[:1000000]
print(len(tabner_mean_toks_smaller))

1000000


In [16]:
stats.ttest_ind(tabner_mean_toks_smaller, wikigs_mean_toks, equal_var=False)

TtestResult(statistic=105.86637750711375, pvalue=0.0, df=1183372.618911697)

### Calculate average num of labeled entities in Wiki-TabNER

In [19]:
data_path="../data/Wiki_TabNER_final_labeled.json"
with open(data_path, 'r') as f:
    ner_tables = json.load(f) 

In [20]:
total_num_cells = 0
total_num_ents = 0
for i in range(len(ner_tables)):
    
    num_cells = len(ner_tables[i][0][6])
    total_num_cells += num_cells
    
    for k in range(len(ner_tables[i][0][6])):
        num_ents = len(ner_tables[i][0][6][k][0])
        total_num_ents+=num_ents

In [21]:
print("Average num of labeled entities per cell {}".format(total_num_ents/total_num_cells))

Average num of labeled entities per cell 2.0061203005619475
